In [ ]:
import numpy as np
import random
import math
from math import exp

In [ ]:
sudokuPuzzle = np.array([[0,0,0,0,3,0,6,0,0],
                         [5,0,0,9,0,0,4,0,0],
                         [0,8,0,6,0,7,0,0,9],
                         [0,7,0,0,0,0,8,0,1],
                         [0,5,0,0,8,0,0,2,0],
                         [3,0,8,0,0,0,0,5,0],
                         [1,0,0,8,0,4,0,9,0],
                         [0,0,2,0,0,6,0,0,5],
                         [0,0,9,0,1,0,0,0,0]])
box_size = int(math.sqrt(len(sudokuPuzzle)))
#sudokuPuzzle

In [ ]:
def copy_grid(sudoku):
    copiedPuzzle = []
    for i in range(len(sudoku)):
        copiedPuzzle.append(list(sudoku[i]))
    return copiedPuzzle
#copy_grid(sudokuPuzzle)

In [ ]:
def print_grid(grid):
    for num in grid:
        print(num)

In [ ]:
def square_coordinates(sudoku, r, c):
    square = []
    row = []
    square_size = int(math.sqrt(len(sudoku)))
    for i in range((r - 1) * square_size, square_size + (r - 1) * square_size  ):
        for j in range((c - 1) * square_size, square_size + (c - 1) * square_size ):
            row.append(sudoku[i][j])
        square.append(row)
        row = []
    return square
            
    
#square_coordinates(sudokuPuzzle, 3, 3)
    

In [ ]:
def nonFixedPositions(sudoku, r, c):
    emptyIndices = []
    square_size = int(math.sqrt(len(sudoku)))
    square = square_coordinates(sudoku, r, c)
    for i in range(square_size):
        for j in range(square_size):
            if(square[i][j] == 0):
                emptyIndices.append([i, j])
    return emptyIndices
#nonFixedPositions(sudokuPuzzle, 1, 2)

In [ ]:
def pickRandomUnfixedPosition(sudoku, i, j):
    randomIndex = random.randint(0, len(nonFixedPositions(sudoku, i, j)) - 1)
    return nonFixedPositions(sudoku, i, j)[randomIndex]
pickRandomUnfixedPosition(sudokuPuzzle, 1, 2)

In [ ]:
def possibleNumsList(sudoku, i, j):
    nonPossibilities = []
    possibilities = []
    square = square_coordinates(sudoku, i, j)
    for row in square:
        for num in row:
            if (num != 0):
                nonPossibilities.append(num)
    for n in range(1, len(sudoku) + 1):
        if (n not in nonPossibilities):
            possibilities.append(n)
    return possibilities

#possibleNumsList(sudokuPuzzle, 2, 2)

In [ ]:
def fillBox(sudoku, i, j):
    box = square_coordinates(sudoku, i, j)
    listOfPossibilities = possibleNumsList(sudokuPuzzle, i, j)
    notFixedPositions = nonFixedPositions(sudoku, i, j)
    for x in range(len(notFixedPositions)):
       box[notFixedPositions[x][0]][notFixedPositions[x][1]] = listOfPossibilities[x]
    return np.array(box)
#fillBox(sudokuPuzzle, 1, 2)

In [ ]:
def fillAllBoxes(sudoku):
    A = []
    B = []
    square_size = int(math.sqrt(len(sudoku)))
    for i in range(1, square_size + 1):
        for j in range(1, square_size + 1):
            A.append(fillBox(sudoku, i, j))

###copied this part completely cause nahi ho raha tha, we need to 
###figure a better way to do this.
    for i in range(3):    
        B.append(np.concatenate((A[0][i], A[1][i], A[2][i]), axis=None).tolist())
    for i in range(3):    
        B.append(np.concatenate((A[3][i], A[4][i], A[5][i]), axis=None).tolist())
    for i in range(3):    
        B.append(np.concatenate((A[6][i], A[7][i], A[8][i]), axis=None).tolist())
    return B
print_grid(fillAllBoxes(sudokuPuzzle))
print_grid(sudokuPuzzle)


In [ ]:
filledSudoku = fillAllBoxes(sudokuPuzzle)
filledSudokuCopy = copy_grid(filledSudoku)
#filledSudoku

In [ ]:
##Cost is the numbers that are missing from the line
"""for example in line 1 [1, 2, 3, 1, 3, 2, 6, 1, 2]
numbers 4, 5, 7, 8, 9 are missing, so cost = 5

"equate cost to the number of
constraint violations"
"""
def costOfRow(sudoku):
    costList = []
    for i in range(len(sudoku)):
        row = sudoku[i]
        cost = 0
        for num in range(len(row)):
            if row.count(num + 1) == 0:
                cost += 1
        costList.append(cost)
    return sum(costList)
costOfRow(filledSudoku)

In [ ]:
def costOfColumn(sudoku):
    costList = []
    transposedSudoku = np.transpose(sudoku)
    for i in range(len(sudoku)):
        col = transposedSudoku[i]
        cost = 0
        for num in range(len(col)):
            if col.tolist().count(num + 1) == 0:
                cost += 1
        costList.append(cost)
    return sum(costList)
costOfColumn(filledSudoku)

In [ ]:
def totalCost(sudoku):
    return costOfRow(sudoku) + costOfColumn(sudoku)
totalCost(filledSudoku)

In [ ]:
def changeCoordinates(i1, j1, i2, j2):
    return [(i1 - 1) * box_size + i2 , (j1 - 1) * box_size + j2]
#changeCoordinates(2,2,0,3)

In [ ]:
def swapCells(sudoku):
    swapped_sudoku = copy_grid((sudoku))
    sudoku_length = len(sudoku)
    box_length = int((sudoku_length) ** (1/2.0))
    #choose a random box
    box_coordinate_1 = random.randint(1, box_length)
    box_coordinate_2 = random.randint(1, box_length)
    #box = square_coordinates(sudoku, box_coordinate_1, box_coordinate_2)
    unfixedPositionList = nonFixedPositions(sudokuPuzzle, box_coordinate_1, box_coordinate_2)

    #choose 2 random cells in non fixed positions of randomly selected box
    coordinate1 = random.randint(0, len(unfixedPositionList) - 1)
    cell1_coordinates = unfixedPositionList[coordinate1]
    coordinate2 = random.randint(0, len(unfixedPositionList) - 1)
    while(coordinate1 == coordinate2):
        coordinate2 = random.randint(0, len(unfixedPositionList) - 1)
    cell2_coordinates = unfixedPositionList[coordinate2]

    revised_cell1_coordinates = changeCoordinates(box_coordinate_1, box_coordinate_2, cell1_coordinates[0], cell1_coordinates[1])
    revised_cell2_coordinates = changeCoordinates(box_coordinate_1, box_coordinate_2, cell2_coordinates[0], cell2_coordinates[1])
    # print(revised_cell1_coordinates)
    # print(revised_cell2_coordinates)
    # print_grid(swapped_sudoku)
    # print()
    #performing the swap
    swapped_sudoku[revised_cell1_coordinates[0]][revised_cell1_coordinates[1]], swapped_sudoku[revised_cell2_coordinates[0]][revised_cell2_coordinates[1]] = \
    swapped_sudoku[revised_cell2_coordinates[0]][revised_cell2_coordinates[1]], swapped_sudoku[revised_cell1_coordinates[0]][revised_cell1_coordinates[1]]
    # print_grid(swapped_sudoku)
    # print()
    return swapped_sudoku 
swapCells(filledSudokuCopy)
# print_grid(filledSudokuCopy)

In [ ]:
# Simulated annealing performs the latter using what’s called an acceptance probability. 
# The acceptance probability is used to determine whether we want to move to a neighbouring state or not. 
# There are a few basic properties of the acceptance probability.

# If the neighbouring state has lesser or equal cost, then we will always move to it.

# If it has greater cost, we will only move to it with a certain probability. 
# Otherwise we’ll stay where we are and choose another neighbour. 
# We define the cost delta denoted by Δc, which is simply the current cost subtracted from the neighbour cost, 
# and a parameter t, which stands for temperature, and influences how likely we are to accept the neighbouring state.

In [ ]:
#anneal algo (STILL NEED TO UNDERSTAND)
def recocidoSimulado(sudo):        
    sudoCopy = copy_grid(sudo)        
    costs = 0    
    temp = 150 
    while temp > 1 :        
        changeProb = 0        
        iterations = 2000
        for i in range(iterations) :            
            if totalCost(sudoCopy) == 0:
                print("Sudoku solved")
                return sudoCopy                
            sudoAfter = swapCells(sudoCopy)
            delta = totalCost(sudoAfter) - totalCost(sudoCopy)            
            if (delta <= 0):
                sudoCopy = sudoAfter
            else :
                uniformValue = random.uniform(0, 1)
                if (uniformValue <= exp(-30 * delta/temp)):
                    #changeProb += 1
                    sudoCopy = sudoAfter            
            costs = (totalCost(sudoCopy))
            print(costs)
        temp = 0.9 * temp  
    return sudoCopy
recocidoSimulado(filledSudoku)

In [ ]:
solution = recocidoSimulado(filledSudoku)
print_grid(solution)